In [1]:
import glob
import numpy as np
import pandas as pd
import seaborn as sns
#import src.visuals as vs
from matplotlib import pyplot as plt
#from IPython.display import display # Allows the use of display() for DataFrames
from scipy.stats import norm
# Import supplementary visualizations code visuals.py
#import lightgbm as lgb
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from collections import Counter

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
#from lightgbm.sklearn import LGBMClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score,fbeta_score,make_scorer,roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.cm as cm
import pickle

from sklearn.externals import joblib
seed = 3

In [2]:
bc = pd.read_csv('data/branded_content_events.csv')
ai = pd.read_csv('data/app_install_events.csv')
ck = pd.read_csv('data/checkout_events.csv')

### data quality check

In [3]:
bc.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2024077 entries, 0 to 2024076
Data columns (total 4 columns):
platform         2024077 non-null object
campaign_name    2024077 non-null object
event_time       2024077 non-null int64
uid              2024077 non-null object
dtypes: int64(1), object(3)
memory usage: 61.8+ MB


In [4]:
bc = bc.drop_duplicates()

In [5]:
bc.rename(columns={'event_time': 'ad_event_time'}, inplace=True)

In [12]:
bc.shape

(2023418, 4)

In [ ]:
bc['event_date_time'] = pd.to_datetime(bc['event_time'],unit='s')
bc['event_date'] = bc['event_date_time'].dt.date

In [ ]:
bc.head()

In [ ]:
bc['evet_dow'] = bc['event_date_time'].dt.dayofweek

In [ ]:
ad_cnt = bc.groupby('event_date')['uid'].count().reset_index().sort_values(by='event_date')

In [ ]:
plt.plot(ad_cnt['event_date'],ad_cnt['uid'],'o--')
plt.title('uid count over advertise time')
plt.xlabel('ad date')
plt.ylabel('Number of uid')

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.plot(ad_cnt['event_date'],ad_cnt['uid'],'o--')
ax.set_title('ads count')
ax.set_xlabel('ads date')
fig.autofmt_xdate(rotation=90)
#plt.xticks(x, labels, rotation='vertical')
ax.set_ylabel('Number of ads')

In [ ]:
bc.head()

In [25]:
bc['platform'].value_counts()

iPhone     1381740
Android     641678
Name: platform, dtype: int64

In [ ]:
top_5_list = bc['campaign_name'].value_counts().nlargest(5).index.tolist()

In [ ]:
bc['campaign_name'] = np.where(bc['campaign_name'].isin(top_5_list),bc['campaign_name'],'non_top_5')

In [ ]:
sign_week_perf = bc.groupby(['day_of_week'],as_index=False)['target'].mean()
fig, ax = plt.subplots()
plt.bar(sign_week_perf['day_of_week'], sign_week_perf['target'])

In [26]:
bc['uid'].nunique()

1690513

In [ ]:
bc[['uid','campaign_name','platform']].drop_duplicates().shape

In [ ]:
bc = bc.sort_values(by='event_time').drop_duplicates(subset=['uid','campaign_name','platform'], keep='last')

In [ ]:
### App Install

In [6]:
ai.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143149 entries, 0 to 143148
Data columns (total 5 columns):
uid           62213 non-null object
device_id     143149 non-null object
event_time    143149 non-null int64
article_id    62213 non-null float64
source        143149 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 5.5+ MB


In [7]:
ai = ai.drop_duplicates()

In [8]:
ai.rename(columns={'event_time': 'ai_event_time'}, inplace=True)

In [ ]:
ai_uid_isnull = ai[ai['uid'].isnull()]
ai_uid_notnull = ai[ai['uid'].notnull()]

In [ ]:
ai_uid_isnull['source'].value_counts()

In [ ]:
ai_uid_notnull['source'].value_counts()

In [ ]:
ai.info(null_counts=True)

In [ ]:
ai['source'].value_counts()

In [ ]:
ai_uid_null = ai[ai['uid'].isnull()]

In [ ]:
ai_uid_null['source'].value_counts()

In [ ]:
ai['']

In [ ]:
### checkout

In [9]:
ck.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17334 entries, 0 to 17333
Data columns (total 4 columns):
device_id          17186 non-null object
checkout_time      17334 non-null int64
checkout_status    17334 non-null object
trial_converted    16303 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 541.8+ KB


In [18]:
ck = ck.drop_duplicates()

In [20]:
ck['checkout_status'].unique()

array(['trial_started', 'direct_purchase'], dtype=object)

In [21]:
ck['trial_converted'].unique()

array([ 1.,  0., nan])

In [ ]:
ck = ck.dropna(axis='device_id')

In [ ]:
ck.rename(columns={'event_time': 'ck_event_time'}, inplace=True)

In [ ]:
### super join

In [10]:
t1 = pd.merge(bc, ai, on='uid', how='left')
t2 = pd.merge(t1, ck, on='device_id', how='left')

In [11]:
t2.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289984515 entries, 0 to 289984514
Data columns (total 11 columns):
platform           289984515 non-null object
campaign_name      289984515 non-null object
ad_event_time      289984515 non-null int64
uid                289984515 non-null object
device_id          65095 non-null object
ai_event_time      65095 non-null float64
article_id         65095 non-null float64
source             65095 non-null object
checkout_time      289923045 non-null float64
checkout_status    289923045 non-null object
trial_converted    276210560 non-null float64
dtypes: float64(4), int64(1), object(6)
memory usage: 25.9+ GB


In [ ]:
# What can you tell us about the effectiveness of our ad campaigns in terms of generating paid subscriptions?
# Ans: number of ads VS number of subscribers

In [14]:
uid_bc_lst = list(bc['uid'].unique())

In [17]:
did_ai_lst = list(ai.loc[ai['uid'].isin(uid_bc_lst),'device_id'].unique())

In [23]:
ck_ad = ck[ck['device_id'].isin(did_ai_lst)]
ck_ad['acquired'] = np.where( (ck_ad['checkout_status'] == 'direct_purchase'), 1, 
         np.where( (ck_ad['checkout_status'] == 'trial_started') & (ck_ad['trial_converted'] == 1), 1, 0) 
        )

/Users/dwightliu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
ck_ad['acquired'].sum()

1008

In [27]:
100*1008/2023418

0.04981669630298831

In [ ]:
#What is our general time to acquisition from seeing the ad to getting a paid subscriber? 
#(Assume trial conversion always takes exactly 7 days)

In [28]:
ck_acq = ck_ad[ck_ad['acquired'] == 1].copy()

In [29]:
ck_acq['acquired_type'] = np.where((ck_acq['checkout_status'] == 'direct_purchase'), 'direct', 'through_trial')

In [30]:
ck_acq = ck_acq[['device_id','checkout_time','acquired_type']].copy()

In [31]:
ai_need = ai[['uid','device_id']].drop_duplicates().copy()

In [35]:
bc.columns

Index(['platform', 'campaign_name', 'ad_event_time', 'uid'], dtype='object')

In [36]:
bc_need = bc[['uid','ad_event_time']].drop_duplicates().sort_values(by='ad_event_time').drop_duplicates(subset='uid', keep='last')

In [37]:
m1 = pd.merge(ck_acq, ai_need, on='device_id', how='inner')

In [38]:
m2 = pd.merge(m1, bc_need, on='uid', how='inner')

In [41]:
m2.columns

Index(['device_id', 'checkout_time', 'acquired_type', 'uid', 'ad_event_time'], dtype='object')

In [44]:
m2['checkout_time'] = pd.to_datetime(m2['checkout_time'])
m2['ad_event_time'] = pd.to_datetime(m2['ad_event_time'])
m2['day_to_acq'] = m2['checkout_time']-m2['ad_event_time']
m2['day_to_acq'] = m2['day_to_acq'].dt.days

In [45]:
m2['day_to_acq'] = np.where(m2['acquired_type'] == 'through_trial', m2['day_to_acq'] + 7, m2['day_to_acq'])

In [46]:
m2['day_to_acq'].mean()

6.521825396825397

In [47]:
maxima = bc.groupby('uid')['ad_event_time'].max()
bc['max_event'] = bc['uid'].map(maxima)

In [48]:
check = bc[bc['max_event'] == bc['ad_event_time']]

In [49]:
check.shape

(1690514, 5)

In [50]:
check['uid'].nunique()

1690513

In [51]:
check[check['uid'].duplicated()]

,platform,campaign_name,ad_event_time,uid,max_event
1261760,Android,author_35,1552951651,0ec028,1552951651


In [52]:
bc[bc['uid'] == '0ec028']

,platform,campaign_name,ad_event_time,uid,max_event
612463,Android,author_88,1552083650,0ec028,1552951651
640633,Android,author_88,1551924319,0ec028,1552951651
660898,Android,author_88,1552046496,0ec028,1552951651
691501,Android,author_88,1552261694,0ec028,1552951651
723754,Android,author_88,1552059757,0ec028,1552951651
870617,Android,author_88,1552071411,0ec028,1552951651
894378,Android,author_88,1552390274,0ec028,1552951651
906340,Android,author_88,1552268507,0ec028,1552951651
1018421,Android,author_35,1552671125,0ec028,1552951651
1041944,Android,author_88,1552321550,0ec028,1552951651


In [ ]:
'''
#Business goal
We want to expand the spend on the sponsored content strategy while maintaining a level of efficiency. 

#The Questions

-	What can you tell us about the effectiveness of our ad campaigns in terms of generating paid subscriptions?
-	What is our general time to acquisition from seeing the ad to getting a paid subscriber? (Assume trial conversion always takes exactly 7 days)
-	What are the factors that affect the conversion rate to a paid subscription and what do you think that tells us about these visitors?

#Additional Discussion
-	What would you recommend the performance marketing manager do? What would you recommend the product manager do? 
-	What additional information would make this analysis more “complete” or actionable? If given that information and more time, how would that change your analysis?
'''